In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

tf.set_random_seed(777)

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
"collect wood and don't assign them tasks and work, but rather "
"teach them to long for the endless immensity of the sea.")

In [3]:
idx2char = list(set(sentence))
char2idx = {c: i for i, c in enumerate(idx2char)}

# hyper parameter
dic_size = len(char2idx)
hidden_size = len(char2idx)
num_classes = len(char2idx)
sequence_length = 10 # any arbitary number
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    
    x = [char2idx[c] for c in x_str]
    y = [char2idx[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)
        
batch_size = len(dataX)

In [4]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])
    
# RNN
x_one_hot = tf.one_hot(X, num_classes)

def lstm_cell():
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cell = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)
outputs, _states = tf.nn.dynamic_rnn(multi_cell, x_one_hot, dtype=tf.float32)

In [5]:
# FC
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(501):
    _, l, results = sess.run(
        [train, loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        if i % 50 == 0:
            print(i, j, ''.join([idx2char[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([idx2char[t] for t in index]), end='')
    else:
        print(idx2char[index[-1]], end='')


0 0 bbuunnnsss 3.219704
0 1 luuggggsss 3.219704
0 2 'onggossss 3.219704
0 3 ongngsssss 3.219704
0 4 nnn.sssuen 3.219704
0 5 hhhouuuu.n 3.219704
0 6 ''ssuu'nnn 3.219704
0 7 ttbiiinnnn 3.219704
0 8 mpeeennnnn 3.219704
0 9 piennnnnnn 3.219704
0 10 .n..nnnnnn 3.219704
0 11 ''nnnnnnnn 3.219704
0 12 ..nnnnnnnn 3.219704
0 13 nnnnnnnnnn 3.219704
0 14 'nnouunuun 3.219704
0 15 nggunnnnnn 3.219704
0 16 houunnnnnn 3.219704
0 17 buen''nnnn 3.219704
0 18 ndnd'nnnnn 3.219704
0 19 fff'nnnnnn 3.219704
0 20 '''nnnoooo 3.219704
0 21 m'nnnnooon 3.219704
0 22 'nnnnnnonn 3.219704
0 23 nnnyynnnnn 3.219704
0 24 ubbboonnne 3.219704
0 25 bbb''rneee 3.219704
0 26 iooosseeee 3.219704
0 27 gganeeeeen 3.219704
0 28 'fnneenenn 3.219704
0 29 fnneennnnn 3.219704
0 30 nnnenennnn 3.219704
0 31 pli''nnnnh 3.219704
0 32 ot'fmmmmhh 3.219704
0 33 .nfmmhhhhh 3.219704
0 34 'fnmhfhhho 3.219704
0 35 ffmffhfhmm 3.219704
0 36 mhhhhhhooo 3.219704
0 37 hhhhhhooos 3.219704
0 38 ffffrrrrss 3.219704
0 39 'hooooosss 3.219704
0 40 httto

100 0 g you want 0.2602077
100 1  you want  0.2602077
100 2 tou want t 0.2602077
100 3 ou want to 0.2602077
100 4 n want to  0.2602077
100 5 pwant to b 0.2602077
100 6 tont to bu 0.2602077
100 7 ont to bui 0.2602077
100 8 nd to buil 0.2602077
100 9 d to build 0.2602077
100 10 hdo build  0.2602077
100 11 to luild a 0.2602077
100 12 h build a  0.2602077
100 13 nluild a s 0.2602077
100 14 tuild a sh 0.2602077
100 15 uild a shi 0.2602077
100 16 pld a ship 0.2602077
100 17 gd a ship, 0.2602077
100 18 e a ship,  0.2602077
100 19  anship, d 0.2602077
100 20 tnship, do 0.2602077
100 21 nship, don 0.2602077
100 22 thip, don' 0.2602077
100 23  ip, don't 0.2602077
100 24 ep, don't  0.2602077
100 25 g, don't d 0.2602077
100 26   don't dr 0.2602077
100 27  bon't dru 0.2602077
100 28 ton't drum 0.2602077
100 29  n't drum  0.2602077
100 30 n't drum u 0.2602077
100 31 dt arum up 0.2602077
100 32 t arum up  0.2602077
100 33 hdrum up p 0.2602077
100 34 toum up pe 0.2602077
100 35  um up peo 0.2602077
10

200 0 m you want 0.2349241
200 1  you want  0.2349241
200 2 tou want t 0.2349241
200 3 ou want to 0.2349241
200 4 n want to  0.2349241
200 5 pwant to b 0.2349241
200 6 tont to bu 0.2349241
200 7 ont to bui 0.2349241
200 8 nd to buil 0.2349241
200 9 d to build 0.2349241
200 10 hdo build  0.2349241
200 11 to build a 0.2349241
200 12 h build a  0.2349241
200 13 nbuild a s 0.2349241
200 14 tuild a sh 0.2349241
200 15 uild a shi 0.2349241
200 16 pld a ship 0.2349241
200 17 md a ship, 0.2349241
200 18 e a ship,  0.2349241
200 19  anship, d 0.2349241
200 20 tnship, do 0.2349241
200 21 nship, don 0.2349241
200 22 thip, don' 0.2349241
200 23 sip, don't 0.2349241
200 24 ep, don't  0.2349241
200 25 m, don't d 0.2349241
200 26 e don't dr 0.2349241
200 27  bon't dru 0.2349241
200 28 ton't arum 0.2349241
200 29  n't drum  0.2349241
200 30 n't drum u 0.2349241
200 31 dt drum up 0.2349241
200 32 t arum up  0.2349241
200 33 hdrum up p 0.2349241
200 34 toum up pe 0.2349241
200 35  um up peo 0.2349241
20

300 0 g you want 0.2320251
300 1  you want  0.2320251
300 2 tou want t 0.2320251
300 3  u want to 0.2320251
300 4 n want to  0.2320251
300 5 twant to b 0.2320251
300 6 tont to bu 0.2320251
300 7 ont to bui 0.2320251
300 8 nd to buil 0.2320251
300 9 d to build 0.2320251
300 10 hao build  0.2320251
300 11 th cuild a 0.2320251
300 12 h cuild a  0.2320251
300 13 nluild a s 0.2320251
300 14 tuild a sh 0.2320251
300 15 uild a shi 0.2320251
300 16 tld a ship 0.2320251
300 17 gd a ship, 0.2320251
300 18 e a ship,  0.2320251
300 19  a ship, d 0.2320251
300 20 tnship, do 0.2320251
300 21 nship, don 0.2320251
300 22 thip, don' 0.2320251
300 23 iip, don't 0.2320251
300 24 ep, don't  0.2320251
300 25 g, don't d 0.2320251
300 26   don't dr 0.2320251
300 27  bon't dru 0.2320251
300 28 ton't drum 0.2320251
300 29  n't drum  0.2320251
300 30 n't drum u 0.2320251
300 31 dt drum up 0.2320251
300 32 t arum up  0.2320251
300 33 harum up p 0.2320251
300 34 toum up pe 0.2320251
300 35  um up peo 0.2320251
30

400 0 g you want 0.23026855
400 1  you want  0.23026855
400 2 tou want t 0.23026855
400 3  u want to 0.23026855
400 4 n want to  0.23026855
400 5 iwant to b 0.23026855
400 6 tont to bu 0.23026855
400 7 ont to bui 0.23026855
400 8 nd to buil 0.23026855
400 9 d to build 0.23026855
400 10  ao build  0.23026855
400 11 to cuild a 0.23026855
400 12   cuild a  0.23026855
400 13 ncuild a s 0.23026855
400 14 tuild a sh 0.23026855
400 15 uild a shi 0.23026855
400 16 ild a ship 0.23026855
400 17 gd a ship, 0.23026855
400 18 e a ship,  0.23026855
400 19  anship, d 0.23026855
400 20 tnship, do 0.23026855
400 21 nship, don 0.23026855
400 22 thip, don' 0.23026855
400 23 iip, don't 0.23026855
400 24 ep, don't  0.23026855
400 25 g, don't d 0.23026855
400 26 , don't dr 0.23026855
400 27  don't dru 0.23026855
400 28 ton't arum 0.23026855
400 29  n't arum  0.23026855
400 30 n't drum u 0.23026855
400 31 dt arum up 0.23026855
400 32 t arum up  0.23026855
400 33  arum up p 0.23026855
400 34 toum up pe 0.2302

500 0 f you want 0.22984947
500 1  you want  0.22984947
500 2 tou want t 0.22984947
500 3  u want to 0.22984947
500 4   want to  0.22984947
500 5 pwant to b 0.22984947
500 6 tont to bu 0.22984947
500 7 ont to bui 0.22984947
500 8 nd to buil 0.22984947
500 9 d to build 0.22984947
500 10 hwo build  0.22984947
500 11 to build a 0.22984947
500 12 h build a  0.22984947
500 13  build a s 0.22984947
500 14 tutld a sh 0.22984947
500 15 utld a shi 0.22984947
500 16 pld a ship 0.22984947
500 17 fd a ship, 0.22984947
500 18 e a ship,  0.22984947
500 19  anship, d 0.22984947
500 20 tnship, do 0.22984947
500 21 nship, don 0.22984947
500 22 thip, don' 0.22984947
500 23 sip, don't 0.22984947
500 24 ep, don't  0.22984947
500 25 f, don't d 0.22984947
500 26 e don't dr 0.22984947
500 27  don't dru 0.22984947
500 28 ton't drum 0.22984947
500 29  n't drum  0.22984947
500 30  't drum u 0.22984947
500 31 dt drum up 0.22984947
500 32 t drum up  0.22984947
500 33 hwrum up p 0.22984947
500 34 toum up pe 0.2298